In [1]:
Experiments_file='/home/kate/Research/YearBuilt/Experiments/DwellingExperiments.xlsx'
AllExperiments_tab='Experiments'
Experiment_name='PartDepFWC'
#Experiment configuration:  features we want to produce PartialDependency
#each line in the file contains a feature name and a type:Categorical or Continuous
#Categorical features values will be process all as is. Continuous only sorted(np.linspace(np.percentile(dataset_temp[f], 0.1),np.percentile(dataset_temp[f], 99.5),50))
Experiment_Features_tab='%s Features'%Experiment_name
#Features Values Input Data: output of Create dataset for PartialDependency:one line with S3 folder with data for all models
Experiment_InputData_tab='%s InputData'%Experiment_name
#Predicted data:each line is a Model name and folder with prediction files. File name is a feature name
Experiment_PredictedData_tab='%s PredictedData'%Experiment_name
#The full set of features the tested model is based on
#each line in the file contains the model name and set of features to built a dataset for SageMaker
#only one,first row is taken into account
Experiment_Models_tab='%s Models'%Experiment_name

#Looks like Trial name should be unique in my environment, not in the experiment it belongs to
Trial_name='%s-PostProcessing'%Experiment_name
 
bucket='kdproperty'
path_to_data_file='/Data/'


instance_type='ml.t3.2xlarge'
instance_count=1

In [2]:
import boto3
import sys
import time
import pandas as pd
import numpy as np
#to read data from S3 with pandas
import s3fs


import sagemaker
from sagemaker import get_execution_role
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput

from sagemaker.analytics import ExperimentAnalytics

In [3]:
region = boto3.session.Session().region_name
role = 'arn:aws:iam::757107622481:role/service-role/AmazonSageMaker-ExecutionRole-20200819T131882'
sagemaker_session = sagemaker.session.Session(default_bucket=bucket)
s3 = s3fs.S3FileSystem()

In [4]:
sys.path.append('/home/kate/Research/YearBuilt/Notebooks/Property')
import ExperimentsUtils as eu

In [5]:
#delete experiment and trials but not output files associated with jobs in experiments
#eu.cleanup_experiment(Experiment_name)
eu.cleanup_trial(Experiment_name, Trial_name)

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [6]:
eu.create_experiment(Experiment_name)
eu.create_trial(Experiment_name,Trial_name)

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [7]:
experiments = pd.read_excel(open(Experiments_file, 'rb'), sheet_name=AllExperiments_tab)
target=experiments[experiments['Experiment']==Experiment_name]['Target'].values[0]
print(target)
data_file=experiments[experiments['Experiment']==Experiment_name]['Dataset'].values[0]
print(data_file)

hasclaim_water
dwelling_basedata_v4.csv


In [8]:
#take into account only one,first row 
#all models have the same dataset predicted from with teh same structure
models = pd.read_excel(open(Experiments_file, 'rb'), sheet_name=Experiment_Models_tab)
for index, row in models.iterrows():
    model_complete_featureset=row[1:51].tolist()
    model_complete_featureset=[x for x in model_complete_featureset if str(x) != 'nan']
    break
model_complete_featureset

['usagetype_encd',
 'cova_deductible',
 'cova_limit ',
 'sqft',
 'cal_year - yearbuilt',
 'landlordind',
 'water_risk_3_blk',
 'constructioncd_encd']

In [9]:
Features = pd.read_excel(open(Experiments_file, 'rb'), sheet_name=Experiment_Features_tab)
Features

,Feature,Type
0,cal_year - yearbuilt,Continuous


In [10]:
PredictedData = pd.read_excel(open(Experiments_file, 'rb'), sheet_name=Experiment_PredictedData_tab)
PredictedData

,Model,DisplayName,Data
0,FWaterClaims,PartDepFWC-Inference-FWaterClaims,s3://kdproperty/PartialDependency/FWaterClaims


In [11]:
InputData = pd.read_excel(open(Experiments_file, 'rb'), sheet_name=Experiment_InputData_tab)
InputData

,Trial Component,Data
0,PartDepFWC-PreparingData,s3://kdproperty/Data/PartialDependencyInputDat...


In [12]:
#input data for post processing
#1.Main dataset
inputs=[ProcessingInput(source='s3://%s%s'%(bucket,path_to_data_file+data_file),
                        destination='/opt/ml/processing/input')]

#2.preprocessed features data for inference 
#need only 1st row for all models
#theye are predicted from teh same dataset
for index, row in InputData.iterrows():
    inputs.append(ProcessingInput(source=row['Data'],
                        destination='/opt/ml/processing/input/InputData'))
    break


#3.inference (prediction) data from each model
for index, row in PredictedData.iterrows():
    inputs.append(ProcessingInput(source=row['Data'],
                        destination='/opt/ml/processing/input/PartialDependency/%s'%row['Model'].replace('_','-')))

In [13]:
%%writefile postprocessingPartialDependencyData.py

#The code joins InputData files for each feature and inference from each model and then average by each feature value


import argparse
import os
import pandas as pd
import numpy as np


if __name__=='__main__':
    
    parser = argparse.ArgumentParser()
    parser.add_argument('--data_file', type=str)
    parser.add_argument('--models', type=str)      
    parser.add_argument('--model_complete_featureset', type=str)    
    parser.add_argument('--featureset', type=str)   
    parser.add_argument('--featuretypes', type=str)        
    args, _ = parser.parse_known_args()    
    print('Received arguments {}'.format(args))
    
    models=args.models.split(',')
    model_complete_featureset=args.model_complete_featureset.split(',')    
    featureset=args.featureset.split(',')
    featuretypes=args.featuretypes.split(',')
    
    #original data file
    input_data_path = os.path.join('/opt/ml/processing/input', args.data_file)
    output_data_path = os.path.join('/opt/ml/processing/output', 'data.csv', )
    
    
    print('Reading input data from {}'.format(input_data_path))
    dataset = pd.read_csv(input_data_path, error_bad_lines=False, index_col=False)

    #final dataset - average pd by each feature value
    all_fm_pd = pd.DataFrame()
    
    for f,ftype in zip(featureset,featuretypes):
        print('Processing %s'%f)
        if ftype=='Continuous':
            feature_InputData_dataset=pd.DataFrame()
            for j in range(0,5):
                feature_InputData_path=os.path.join('/opt/ml/processing/input/InputData', '%s_%s.csv'%(f,j))
                feature_InputData_dataset_j = pd.read_csv(feature_InputData_path, names=model_complete_featureset, error_bad_lines=False, index_col=False)
                feature_InputData_dataset=feature_InputData_dataset.append(feature_InputData_dataset_j)          
        else:
            feature_InputData_path=os.path.join('/opt/ml/processing/input/InputData', '%s.csv'%f)
            feature_InputData_dataset = pd.read_csv(feature_InputData_path, names=model_complete_featureset, error_bad_lines=False, index_col=False)
        fm_pd = pd.DataFrame()
        for m in models:
            print('Reading predicted data from model %s'%m)
            if ftype=='Continuous':
                pdf_dataset=pd.DataFrame()
                for j in range(0,5):
                    model_predicted_data_path =os.path.join('/opt/ml/processing/input/PartialDependency/%s'%m.replace('_','-'),'%s_%s.csv.out'%(f,j))
                    pdf_dataset_i = pd.read_csv(model_predicted_data_path, names=['pd'], error_bad_lines=False, index_col=False)
                    pdf_dataset=pdf_dataset.append(pdf_dataset_i)
            else:
                model_predicted_data_path =os.path.join('/opt/ml/processing/input/PartialDependency/%s'%m.replace('_','-'),'%s.csv.out'%f)                
                pdf_dataset = pd.read_csv(model_predicted_data_path, names=['pd'], error_bad_lines=False, index_col=False)
            #model feature partial dependency columns name
            pd_column_name='%s_pd'%m
            feature_InputData_dataset[pd_column_name]= pdf_dataset['pd'].values
            
            #average
            fm_s = feature_InputData_dataset.groupby(f)[pd_column_name].mean()
            fm_pd_model=pd.DataFrame({'value':fm_s.index, pd_column_name:fm_s.values})           
            fm_pd_model['feature']=f
            fm_pd_model = fm_pd_model[['feature','value',pd_column_name]]

            fm_pd=pd.concat([fm_pd,fm_pd_model],axis=1)

        #add text value for categorical encd columns
        #assuming there is encoded (_encd ended) and original values in the dataset
        fm_pd['value2']=fm_pd['value'].astype(str)
        if '_encd' in f and f.replace('_encd','') in dataset.columns:
            #unique combindation of codes and original values from the main dataset into list and then dictionary
            dataset['dummy']= dataset[f.replace('_encd','')] +'-'+ dataset[f].astype(str)
            unique_comb_l=dataset['dummy'].unique().tolist()
            unique_comb_value=[i.split('-', 1)[0] for i in unique_comb_l]
            unique_comb_key=[i.split('-', 1)[1] for i in unique_comb_l]
            unique_comb_dict = dict(zip(unique_comb_key, unique_comb_value))
            #replace value2 in the feature values and partial dependencies
            fm_pd['value2'].replace(unique_comb_dict, inplace=True)
        
        #join all features in one dataset    
        all_fm_pd=all_fm_pd.append(fm_pd)
       
    #saving final output    
    all_fm_pd.to_csv(output_data_path,header=True ,index=False)    

Overwriting postprocessingPartialDependencyData.py


In [14]:
processors=list()

featureset=Features['Feature'].tolist()
featuretypes=Features['Type'].tolist()
models=PredictedData['Model'].tolist()

data_processor = SKLearnProcessor(framework_version='0.20.0',
                                     role=role,
                                     instance_type=instance_type,
                                     instance_count=instance_count)

data_processor.run(code='postprocessingPartialDependencyData.py',
                        inputs=inputs,
                        outputs=[ProcessingOutput(output_name='output', source='/opt/ml/processing/output', destination='s3://%s%sPartialDependencyOutputData/%s'%(bucket,path_to_data_file,Experiment_name)),                                                          
                                ],
                        arguments=['--data_file',data_file,
                                 '--models',','.join(models).replace(' ',''),
                                 '--model_complete_featureset',','.join(model_complete_featureset).replace(' ',''),
                                 '--featureset', ','.join(featureset).replace(' ',''),
                                 '--featuretypes', ','.join(featuretypes).replace(' ','')],
                        experiment_config = {
        'ExperimentName': Experiment_name ,
        'TrialName' : Trial_name,
        'TrialComponentDisplayName' : '%s'%(Trial_name,),},
                        wait=False
                        )
processors.append(data_processor)

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Defaulting to only available Python version: py3
INFO:sagemaker:Creating processing-job with name sagemaker-scikit-learn-2020-12-31-20-52-09-867



Job Name:  sagemaker-scikit-learn-2020-12-31-20-52-09-867
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://kdproperty/Data/dwelling_basedata_v4.csv', 'LocalPath': '/opt/ml/processing/input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'input-2', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://kdproperty/Data/PartialDependencyInputData/PartDepFWC', 'LocalPath': '/opt/ml/processing/input/InputData', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'input-3', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://kdproperty/PartialDependency/FWaterClaims', 'LocalPath': '/opt/ml/processing/input/PartialDependency/FWaterClaims', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppMana

In [15]:
#wait till the rest of processing jobs complete
eu.wait_processing_jobs(processors=processors,check_every_sec=10,print_every_n_output=10,wait_min=60)

Processing job sagemaker-scikit-learn-2020-12-31-20-52-09-867 status: InProgress
Continue waiting...
Processing job sagemaker-scikit-learn-2020-12-31-20-52-09-867 status: InProgress
Continue waiting...
Processing job sagemaker-scikit-learn-2020-12-31-20-52-09-867 status: InProgress
Continue waiting...
All Processing Jobs are Completed


In [16]:
trial_component_analytics = ExperimentAnalytics(
    experiment_name=Experiment_name
)
trial_comp_ds = trial_component_analytics.dataframe()
trial_ds=trial_comp_ds[trial_comp_ds['DisplayName'].str.contains(Trial_name)].copy()
trial_ds=trial_ds[['DisplayName','output - Value']]
trial_ds.columns=['Trial Component','Data']
trial_ds

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


,Trial Component,Data
0,PartDepFWC-PostProcessing,s3://kdproperty/Data/PartialDependencyOutputDa...


In [17]:
#Saving into the Experiment log location of the created  dataset
eu.SaveToExperimentLog(Experiments_file, '%s OutputData'%Experiment_name, trial_ds)

In [18]:
#read data from S3 and visualize and save to a log
#header as hardcoded in postprocessingPartialDependencyData.py: feature name, feature value, models partial dependency value (as we cofigured in the experiment), feature original value if encoded
#the output file name (required in this case) is data.csv, hardcoded in postprocessingPartialDependencyData.py
pdf_dataset = pd.read_csv(trial_ds['Data'][0]+'/data.csv',  error_bad_lines=False, index_col=False) 

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [19]:
#Saving into the Experiment log or load to redshift directly
eu.SaveToExperimentLog(Experiments_file, '%s Data'%Experiment_name, pdf_dataset)

In [20]:
pdf_dataset.head()

,feature,value,FWaterClaims_pd,value2
0,cal_year-yearbuilt,0.000000,0.000538,0.000000
1,cal_year-yearbuilt,2.346939,0.000538,2.346939
2,cal_year-yearbuilt,4.693878,0.000599,4.693878
3,cal_year-yearbuilt,7.040816,0.000686,7.040816
4,cal_year-yearbuilt,9.387755,0.001009,9.387755


In [21]:
pdf_dataset['pd']=0
for m in models:
    pdf_dataset['pd'] = pdf_dataset['pd'] + pdf_dataset['%s_pd'%m]
pdf_dataset['pd'] = pdf_dataset['pd']/len(models)

In [22]:
#Visualization Partial Dependency with bokeh

In [23]:
import bokeh
import bokeh.io
bokeh.io.output_notebook()
from bokeh.plotting import  figure, show
from bokeh.models import HoverTool   
class HoverHelper():

    def hovertool(self):
        tooltips = [
            ('pd','@pd'),
            ('value','@value2'),
        ]
        ht = HoverTool(tooltips=tooltips)
        return ht
    def tools(self, standard_tools='pan,crosshair,wheel_zoom,zoom_in,zoom_out,undo,reset'):
        return [self.hovertool(), standard_tools]
hover = HoverHelper()    

Loading BokehJS ...

In [24]:
f='cal_year-yearbuilt'
p = figure(plot_width=900, 
               plot_height=400, 
               tools=hover.tools(),
               x_axis_label='Values',
               y_axis_label='Partial Dependency',
               title='%s Partial Dependency'%f)
p.circle(source=pdf_dataset[pdf_dataset['feature']==f], x='value', y='pd')
show(p)

In [ ]:
f='constructioncd_encd'
values = pdf_dataset[pdf_dataset['feature']==f]['value2'].tolist()


p = figure(plot_width=900, 
               plot_height=400, 
               tools=hover.tools(),
               x_range=values,
               x_axis_label='Values',
               y_axis_label='Partial Dependency',
               title='%s Partial Dependency'%f)
p.vbar(source=pdf_dataset[pdf_dataset['feature']==f], x='value2', top='pd', width=0.70)


show(p)

In [ ]:
f='landlordind'
values = pdf_dataset[pdf_dataset['feature']==f]['value2'].tolist()


p = figure(plot_width=900, 
               plot_height=400, 
               tools=hover.tools(),
               x_range=values,
               x_axis_label='Values',
               y_axis_label='Partial Dependency',
               title='%s Partial Dependency'%f)
p.vbar(source=pdf_dataset[pdf_dataset['feature']==f], x='value2', top='pd', width=0.70)


show(p)